<a href="https://colab.research.google.com/github/KoreTane/PredictiveLeadManagement/blob/main/Forecasting_marketing_investments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import warnings

In [3]:
#dfm = pd.read_excel('MatriculaCEP.xlsx')
#dfc = pd.read_excel('fluxo_caixa_cbs.xlsx')
df = pd.read_excel('CrmAgendaClientes.xlsx')

In [ ]:
# Extrair a coluna a partir do 2 delimitador '#'
#dfc['Matricula'] = dfc['Descrição'].str.split('#', expand=True)[2]
# Remover os caracteres '#\d+' da coluna 'Descrição'
#dfc['Aluno'] = dfc['Descrição'].str.replace(r'#\d+', '', regex=True)
#dfc = dfc[['Matricula', 'Data', 'Aluno', 'Valor', 'Tipo']]
warnings.filterwarnings('ignore')

In [14]:
# Dados dos investimentos em MKT
trimestre = ['1','2','3','4','1','2']
ano = ['2023','2023','2023','2023','2024','2024']
cliques = [303, 1179, 1654, 841, 1666, 812]
impressoes = [9917, 29871, 51773, 27408, 37411, 12657]
cpc_medio = ['0,60', '0,66', '0,55', '0,73', '0,55', '0,44']
custo = ['181,82', '780,94', '917,74', '612,22', '919,50', '357,89']

# Criar DataFrame
dfm = pd.DataFrame({
    'Trimestre': trimestre,
    'ano':ano,
    'Cliques': cliques,
    'Impressões': impressoes,
    'CPC Médio': cpc_medio,
    'Custo': custo
})

In [15]:
dfm

,Trimestre,ano,Cliques,Impressões,CPC Médio,Custo
0,1,2023,303,9917,"0,60","181,82"
1,2,2023,1179,29871,"0,66","780,94"
2,3,2023,1654,51773,"0,55","917,74"
3,4,2023,841,27408,"0,73","612,22"
4,1,2024,1666,37411,"0,55","919,50"
5,2,2024,812,12657,"0,44","357,89"


In [7]:
df['SITUAÇÃO_FINAL'].value_counts()

SITUAÇÃO_FINAL
DESISTIU       1247
MATRICULADO     277
EM ABERTO       142
Name: count, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1666 entries, 0 to 1665
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA 1º CONTATO   1666 non-null   datetime64[ns]
 1   TRIMESTRE         1666 non-null   int64         
 2   NOME COMPLETO     1664 non-null   object        
 3   IDADE             1666 non-null   int64         
 4   FAIXA ETARIA      1666 non-null   object        
 5   GENERO            1666 non-null   object        
 6   TELEFONE          1666 non-null   object        
 7   STATUS            1666 non-null   object        
 8   VIA               1666 non-null   object        
 9   ESTILOS           1666 non-null   object        
 10  DATA AGENDAMENTO  1666 non-null   object        
 11  TURNO             1666 non-null   object        
 12  PROFESSOR         1666 non-null   object        
 13  COMPARECEU        1666 non-null   object        
 14  SITUAÇÃO_FINAL    1666 n

In [ ]:
df[df['DATA'].dt.year == 2023]

In [10]:
# Filtrar o DataFrame para incluir apenas as linhas que correspondem ao trimestre 1 do ano 2023 e ao tipo de contato "1º Contato"
df_filtered = df[(df['TRIMESTRE'] == 1) & (df['DATA 1º CONTATO'].dt.year == 2023)]

# Contar a quantidade de situações finais em cada tipo
situacao_final_counts = df_filtered['SITUAÇÃO_FINAL'].value_counts()

In [11]:
situacao_final_counts

SITUAÇÃO_FINAL
DESISTIU       216
EM ABERTO       40
MATRICULADO     31
Name: count, dtype: int64